# Blackbox testing of robustness against BoundaryAttack


### Importing required packages

In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import tensorflow as tf


from art.attacks import BoundaryAttack
from art.classifiers import KerasClassifier, BlackBoxClassifier
from art.utils import load_mnist

tf.compat.v1.disable_eager_execution()

### Load Dataset

In [17]:
# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

### Creating a sample model
A sample model is used to demonstrate the use of ART library
Model to be tested will be replaced here

In [18]:
# Step 2: Create the model

model = Sequential()

model.add(Conv2D(4,5,padding='same',input_shape = [28, 28, 1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(10,5, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Wrapping of ART classifier
Wrapping is necessary to use ART functions

In [19]:
# Step 3: wrap ART tf classifier
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

Epoch 1/10
469/468 [==============================] - 5s 10ms/step - loss: 0.3697 - accuracy: 0.8914
Epoch 2/10
469/468 [==============================] - 5s 10ms/step - loss: 0.1059 - accuracy: 0.9683
Epoch 3/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0739 - accuracy: 0.9768
Epoch 4/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0567 - accuracy: 0.9821
Epoch 5/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0458 - accuracy: 0.9854
Epoch 6/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0393 - accuracy: 0.9874
Epoch 7/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0327 - accuracy: 0.9895
Epoch 8/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0281 - accuracy: 0.9908
Epoch 9/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0245 - accuracy: 0.9925
Epoch 10/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0217 - accura

### Evaluating model based on original Dataset

In [30]:
# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

Accuracy on benign test examples: 98.83999999999999%


### Creating Adversarial samples
Using Boundary Attack
- We will select a specific set of data from x_test and y_test as time taken to generate adversarial samples for all data from x_test is too long

In [51]:
# Step 6: Generate adversarial test examples
attack = BoundaryAttack(classifier)
x_test_adv = attack.generate(x=x_test[200:201], y=y_test[200:201])

### Evaluating model on adversarial samples

In [52]:
# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

Accuracy on adversarial test examples: 10.100000000000001%


### Evaluation results
- Using accuracy as an evaluation metric, we can see that the sample model's performance dropped by a huge amount after going through adversarial samples.
- Results is normal as sample model does not have defence against adversarial attacks
